In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('data.csv')
len(df)

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [3]:
df = pd.read_csv('./data/data.csv')
len(df)

11914

In [4]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [5]:
np.random.seed(2)

n = len(df)

n_val = int(0.2 * n)
n_test = int(0.2 * n)
n_train = n - (n_val + n_test)

idx = np.arange(n)
np.random.shuffle(idx)

df_shuffled = df.iloc[idx]

df_train = df_shuffled.iloc[:n_train].copy()
df_val = df_shuffled.iloc[n_train:n_train+n_val].copy()
df_test = df_shuffled.iloc[n_train+n_val:].copy()

In [6]:

y_train_orig = df_train.msrp.values
y_val_orig = df_val.msrp.values
y_test_orig = df_test.msrp.values

y_train = np.log1p(df_train.msrp.values)
y_val = np.log1p(df_val.msrp.values)
y_test = np.log1p(df_test.msrp.values)

del df_train['msrp']
del df_val['msrp']
del df_test['msrp']

In [7]:

def linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

In [8]:
base = ['engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg', 'popularity']

In [9]:
def prepare_X(df):
    df_num = df[base]
    df_num = df_num.fillna(0)
    X = df_num.values
    return X

In [10]:
X_train = prepare_X(df_train)
w_0, w = linear_regression(X_train, y_train)

In [11]:
df_new = df.head()

In [12]:
y_pred = w_0 + df_new.dot(w)

ValueError: Dot product shape mismatch, (5, 16) vs (5,)

In [13]:
del df_new['msrp']

In [14]:
X_train = prepare_X(df_new)
y_pred = w_0 + X_train.dot(w)

In [15]:
y_pred

array([10.84639436, 10.53544645, 10.55039056,  9.84108968,  9.84108968])

In [16]:
np.expm1(y_pred)

array([51347.67231413, 37624.84474245, 38191.35182832, 18789.18017145,
       18789.18017145])

In [17]:
df_new[msrp] = np.expm1(y_pred)
df_new[msrp_pred] = y_pred
df_new

NameError: name 'msrp' is not defined

In [18]:
df_new['msrp'] = np.expm1(y_pred)
df_new['msrp_pred'] = y_pred
df_new

<ipython-input-18-b75b26f65a58>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['msrp'] = np.expm1(y_pred)
<ipython-input-18-b75b26f65a58>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['msrp_pred'] = y_pred


,make,model,year,engine_fuel_type,engine_hp,engine_cylinders,transmission_type,driven_wheels,number_of_doors,market_category,vehicle_size,vehicle_style,highway_mpg,city_mpg,popularity,msrp,msrp_pred
0,bmw,1_series_m,2011,premium_unleaded_(required),335.0,6.0,manual,rear_wheel_drive,2.0,"factory_tuner,luxury,high-performance",compact,coupe,26,19,3916,51347.672314,10.846394
1,bmw,1_series,2011,premium_unleaded_(required),300.0,6.0,manual,rear_wheel_drive,2.0,"luxury,performance",compact,convertible,28,19,3916,37624.844742,10.535446
2,bmw,1_series,2011,premium_unleaded_(required),300.0,6.0,manual,rear_wheel_drive,2.0,"luxury,high-performance",compact,coupe,28,20,3916,38191.351828,10.550391
3,bmw,1_series,2011,premium_unleaded_(required),230.0,6.0,manual,rear_wheel_drive,2.0,"luxury,performance",compact,coupe,28,18,3916,18789.180171,9.841090
4,bmw,1_series,2011,premium_unleaded_(required),230.0,6.0,manual,rear_wheel_drive,2.0,luxury,compact,convertible,28,18,3916,18789.180171,9.841090


In [19]:
df_new[base]

,engine_hp,engine_cylinders,highway_mpg,city_mpg,popularity
0,335.0,6.0,26,19,3916
1,300.0,6.0,28,19,3916
2,300.0,6.0,28,20,3916
3,230.0,6.0,28,18,3916
4,230.0,6.0,28,18,3916


In [20]:
del df_new['make','model','year','engine_fuel_type','engine_hp']

KeyError: ('make', 'model', 'year', 'engine_fuel_type', 'engine_hp')

In [21]:
del df_new['make']

In [22]:
del df_new['model']

In [23]:
del df_new['year']

In [24]:
df_new.head

<bound method NDFrame.head of               engine_fuel_type  engine_hp  engine_cylinders transmission_type  \
0  premium_unleaded_(required)      335.0               6.0            manual   
1  premium_unleaded_(required)      300.0               6.0            manual   
2  premium_unleaded_(required)      300.0               6.0            manual   
3  premium_unleaded_(required)      230.0               6.0            manual   
4  premium_unleaded_(required)      230.0               6.0            manual   

      driven_wheels  number_of_doors                        market_category  \
0  rear_wheel_drive              2.0  factory_tuner,luxury,high-performance   
1  rear_wheel_drive              2.0                     luxury,performance   
2  rear_wheel_drive              2.0                luxury,high-performance   
3  rear_wheel_drive              2.0                     luxury,performance   
4  rear_wheel_drive              2.0                                 luxury   

  vehicl

In [25]:
df_new.head()

,engine_fuel_type,engine_hp,engine_cylinders,transmission_type,driven_wheels,number_of_doors,market_category,vehicle_size,vehicle_style,highway_mpg,city_mpg,popularity,msrp,msrp_pred
0,premium_unleaded_(required),335.0,6.0,manual,rear_wheel_drive,2.0,"factory_tuner,luxury,high-performance",compact,coupe,26,19,3916,51347.672314,10.846394
1,premium_unleaded_(required),300.0,6.0,manual,rear_wheel_drive,2.0,"luxury,performance",compact,convertible,28,19,3916,37624.844742,10.535446
2,premium_unleaded_(required),300.0,6.0,manual,rear_wheel_drive,2.0,"luxury,high-performance",compact,coupe,28,20,3916,38191.351828,10.550391
3,premium_unleaded_(required),230.0,6.0,manual,rear_wheel_drive,2.0,"luxury,performance",compact,coupe,28,18,3916,18789.180171,9.841090
4,premium_unleaded_(required),230.0,6.0,manual,rear_wheel_drive,2.0,luxury,compact,convertible,28,18,3916,18789.180171,9.841090
